In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [24]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key='226fb2133a571259cc4b9c5f3e0ae4d4.5P9J31WGgJQVGj6u')  # 请填写您自己的API Key
result = client.knowledge.create(
    embedding_id=3,
    name='wenlin knowledge',
    description='knowledge description'
)
print(result.id)


1862046346182262784


In [26]:
resp = client.knowledge.document.create(
    file=open("关于极限滑雪运动.docx", "rb"),  # 换成自己的文件
    purpose="retrieval",
    knowledge_id="1862046346182262784",
    sentence_size=202,
    custom_separator=["\n"]
)

print(resp)


DocumentObject(successInfos=[DocumentSuccessinfo(documentId='1862046714245042176', filename=None, fileName='关于极限滑雪运动.docx')], failedInfos=[])


In [37]:
import zhipuai

In [ ]:
# 设置提示词（模板）
prompt_template = """
从文档|[knowledge_id]中找问题|[question1]的答案，
找到答案就仅使用文档语句回答，找不到答案就用自身知识回答并告诉用户该信息不是来自文档。
不要复述问题，直接开始回答
"""
 
response = client.chat.completions.create(
    model="gln-4",  # 设置具体大模型是gln-4
    messages=[
        {"role": "user", "content": "未来的情况"}  # 提问，同OpenAI
    ],
    tools=[
        {
            "type": "retrieval",  # 设置工具是“抽取”
            "retrieval": [
                {"knowledge_id": kb_id}  # 设置问答基于的知识库的id
            ],
            "prompt_template": prompt_template,  # 设置提示词
            "stream": False  # 设置不用流式
        }
    ]
)
 
# 输出回答对象
response


In [ ]:
import docx

# 假设你已经将问答数据预处理成了一个字典列表，格式如下：
# questions_answers = [
#     {"question": "什么是极限滑雪？", "answer": "极限滑雪是一种高风险、高技巧的滑雪运动..."},
#     {"question": "极限滑雪有哪些主要类型？", "answer": "极限滑雪主要包括高山滑雪、自由式滑雪、单板滑雪等类型..."},
#     ...
# ]

# 由于直接从.docx读取并解析成这种格式较为复杂，这里提供一个简化的读取和解析函数示例：
def read_questions_answers_from_docx(file_path):
    doc = docx.Document(file_path)
    questions_answers = []
    current_question = ""
    current_answer = ""
    
    for para in doc.paragraphs:
        if para.text.startswith("问："):
            if current_question and current_answer:
                questions_answers.append({"question": current_question.strip("问： "), "answer": current_answer.strip()})
            current_question = para.text
            current_answer = ""
        else:
            current_answer += para.text + "\n"  # 假设答案可能跨越多个段落
    
    # 添加最后一个问答对（如果存在）
    if current_question and current_answer:
        questions_answers.append({"question": current_question.strip("问： "), "answer": current_answer.strip()})
    
    return questions_answers

# 加载问答数据
file_path = "关于极限滑雪运动.docx"
questions_answers = read_questions_answers_from_docx(file_path)

# 问答聊天和搜索功能
def chat_and_search():
    print("欢迎来到极限滑雪运动问答系统！")
    print("你可以输入问题来获取答案，或者输入'退出'来结束聊天。")
    
    while True:
        user_input = input("请输入你的问题：")
        if user_input.lower() == "退出":
            break
        
        # 搜索问题并获取答案
        for qa in questions_answers:
            if user_input.strip().lower() in qa["question"].lower():
                print(f"答案：{qa['answer']}")
                break
        else:
            print("抱歉，没有找到相关问题的答案。")

# 运行问答聊天和搜索功能
chat_and_search()

In [ ]:
from zhipuai import ZhipuAI

# 请填写您自己的API Key
client = ZhipuAI(api_key='226fb2133a571259cc4b9c5f3e0ae4d4.5P9J31WGgJQVGj6u')

# 创建知识库（这一步您已经成功完成了，并得到了知识库ID）
# result = client.knowledge.create(embedding_id=3, name='wenlin knowledge', description='knowledge description')
# print(result.id)  # 假设这一步返回了知识库ID：1862046346182262784

# 设置提示词（模板）
prompt_template = """
从文档|[knowledge_id]中找问题|[question1]的答案，
找到答案就仅使用文档语句回答，找不到答案就用自身知识回答并告诉用户该信息不是来自文档。
不要复述问题，直接开始回答
"""

# 使用正确的知识库ID和retrieval格式
knowledge_id = '1862046346182262784'  # 这里应该是您之前创建知识库时得到的ID
response = client.chat.completions.create(
    model="gln-4",  # 设置具体大模型是gln-4
    messages=[
        {"role": "user", "content": "关于极限滑雪运动未来的情况"}  # 提问
    ],
    tools=[
        {
            "type": "retrieval",  # 设置工具是“抽取”
            "retrieval": {  # 注意这里是一个字典，而不是列表
                "knowledge_id": knowledge_id  # 直接使用知识库ID作为键（如果API要求的是这样）
                # 或者如果API期望的是一个简单的键值对列表（这种情况较少见），则应该是：
                # "retrieval": [{"key": "knowledge_id", "value": knowledge_id}]
                # 但根据错误信息，这似乎不是API所期望的
            },
            "prompt_template": prompt_template,  # 设置提示词
            "stream": False  # 设置不用流式
        }
    ]
)

# 输出回答对象（注意：这里应该处理response对象以获取有用的信息）
# 假设response是一个字典或具有.json()方法的对象（取决于zhipuai库的实现）
print(response.json())  # 如果response对象有.json()方法，则调用它以获取JSON格式的响应
# 或者如果response已经是一个字典，则直接打印它：
# print(response)